# KIVA CAPSTONE PROJECT - DATA CLEANING

These are the cleaning steps I went through to make this dataset suitable for modelling.
*** 

In [1]:
#IMPORTS
from sklearn.preprocessing import StandardScaler

import pandas as pd
import seaborn as sns
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
#LOAD THE DATASET
loans = pd.read_csv('/Users/nicolas/Downloads/loans2.csv',low_memory=False)

In [3]:
#LOWER THE COLUMNS
lower_columns = []
for i in loans.columns:
    lower_columns.append(i.lower())
loans.columns = lower_columns

In [4]:
loans.index

RangeIndex(start=0, stop=1511575, step=1)

## REMOVING NULLS

Let's start with the first step, removing nulls.

In [5]:
loans.isnull().sum()

unnamed: 0                               0
loan_id                                  0
loan_name                            25235
original_language                    21677
description                          21683
description_translated              272437
funded_amount                            0
loan_amount                              0
status                                   0
image_id                             21677
video_id                           1511259
activity_name                            0
sector_name                              0
loan_use                             21686
country_code                            31
country_name                             0
town_name                            97508
currency_policy                          0
currency_exchange_coverage_rate     243071
currency                                 0
partner_id                               0
posted_time                              0
planned_expiration_time             152761
disburse_ti

## DESCRIPTION

In [6]:
loans[(loans.description_translated.isnull()) & (loans.original_language == 'Spanish')].head(5)

,unnamed: 0,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,image_id,video_id,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_names,borrower_genders,borrower_pictured,repayment_interval,distribution_model,time,delinquency,default_rate,rural_pct
368793,368793,843681,Maria Salome,Spanish,"Salome, de 28 años de edad, no estudió, porque...",NaN,650.0,650.0,funded,1819723.0,NaN,Agriculture,Agriculture,NaN,SV,El Salvador,Osicala,shared,0.1,USD,199.0,2015-02-19 19:13:57.000 +0000,2015-03-25 17:00:02.000 +0000,2015-01-29 08:00:00.000 +0000,2015-02-28 08:05:31.000 +0000,15.0,21,2,1,volunteer_pick,Maria Salome,female,true,bullet,field_partner,114,2.61,1.50,87.0
565339,565339,28091,Hilda Yolanda,Spanish,<p>Hilda is asking for a loan to expand her bu...,NaN,325.0,325.0,funded,99567.0,NaN,Food Production/Sales,Food,Business Expansion,SV,El Salvador,"La Libertad, La Libertad",standard,NaN,USD,81.0,2007-12-17 22:21:47.000 +0000,NaN,2008-01-04 17:41:38.000 +0000,2007-12-21 17:41:38.000 +0000,11.0,11,2,1,NaN,Hilda Yolanda,female,true,monthly,field_partner,159,0.00,3.46,57.0
565347,565347,139879,Claudia Marina,Spanish,Claudia Marina R. D. F. resiD. en el munici...,NaN,500.0,500.0,funded,394625.0,NaN,Food Production/Sales,Food,compra de insumos para aumentar producción,SV,El Salvador,"San Salvador, San Salvador",standard,NaN,USD,81.0,2009-09-25 13:11:54.000 +0000,NaN,2009-08-26 07:00:00.000 +0000,2009-09-29 18:27:28.000 +0000,14.0,19,1,1,NaN,Claudia Marina,female,true,monthly,field_partner,159,0.00,3.46,57.0
565420,565420,28123,Maria Elena,Spanish,Mrs. Z. lives with her husband and children in...,NaN,600.0,600.0,funded,99724.0,NaN,Grocery Store,Food,To stock a small grocery store.,SV,El Salvador,"Colonia las Arboledas #2 pasaje 6,Lourdes,Colon",standard,NaN,USD,81.0,2007-12-18 03:30:26.000 +0000,NaN,2008-01-05 19:09:49.000 +0000,2007-12-22 19:09:49.000 +0000,15.0,23,1,1,NaN,Maria Elena,female,true,monthly,field_partner,159,0.00,3.46,57.0
565430,565430,141809,Sonia,Spanish,"Sonia C. M. de 40 años de edad, vive en Apopa ...",NaN,800.0,800.0,funded,399961.0,NaN,Textiles,Arts,Tela para la elaboracion de ropa,SV,El Salvador,"San Salvador, San Salvador",standard,NaN,USD,81.0,2009-10-04 18:32:28.000 +0000,NaN,2009-09-18 07:00:00.000 +0000,2009-10-05 19:43:17.000 +0000,20.0,30,1,1,NaN,Sonia,female,true,monthly,field_partner,159,0.00,3.46,57.0


In [7]:
description_nulls = loans[(loans.description_translated.isnull())]

In [8]:
loans.loc[description_nulls.index,'description_translated'] = description_nulls.description

In [9]:
loans.drop(loans[loans.description_translated.isnull()].index,inplace=True)

In [10]:
loans.drop(columns='description',inplace=True)

In [11]:
new_columns = {'description_translated':'description'}
loans = loans.rename(columns = new_columns)

In [12]:
loans.isnull().sum()

unnamed: 0                               0
loan_id                                  0
loan_name                             3558
original_language                        0
description                              0
funded_amount                            0
loan_amount                              0
status                                   0
image_id                                 0
video_id                           1489576
activity_name                            0
sector_name                              0
loan_use                                 4
country_code                            31
country_name                             0
town_name                            75831
currency_policy                          0
currency_exchange_coverage_rate     239990
currency                                 0
partner_id                               0
posted_time                              0
planned_expiration_time             149965
disburse_time                            0
raised_time

### TOWNS

There was many nulls for the town_name column but 0 nulls for the country_name column. I decided to replace the nulls in the town column by their respective countries, to keep the data truthful, even if cities aren't countries, but it still tells us it's a city inside that country.

In [13]:
#SUBSET 
towns_nulls = loans[loans.town_name.isnull()]

In [14]:
#CREATING LIST OF COUNTRY NAMES FOR THESE NULLS
townlist = towns_nulls.country_name

In [15]:
#USING LOC TO SET THE COUNTRIES UNDER TOWN_NAMES
loans.loc[towns_nulls.index,'town_name'] = townlist

### LOANS CURRENCY EXCHANGE COVERAGE RATE 

This is the column showing if a certain loan has a currency exchange coverage rate policy, or in other words, a compensation for currency fluctuations from the lender. I thought that the nulls arose for 2 reasons. The first is simply that there is no currency exchange policy happening, so no agreement between KIVA and a certain bank, and the second is that there is no need to do so as the currency dealt with is in dollars, meaning no currency exchange, so no problem. This is why I divided the nulls into 2 groups, those that had US dollars as currency and those that didn't.

In [16]:
loans.currency_exchange_coverage_rate.value_counts()

0.1    1096133
0.2      91964
0.0      61805
Name: currency_exchange_coverage_rate, dtype: int64

In [17]:
#SUSBSET 
exchange_nulls = loans[loans.currency_exchange_coverage_rate.isnull()]

In [18]:
#CATEGORISATION OF NULLS
exchanges_list = []
for i in exchange_nulls.currency:
    if i == 'USD':
        exchanges_list.append('US_dollars')
    else:
        exchanges_list.append(0.0)

In [19]:
#CHANGING NULLS USING LOC
loans.loc[exchange_nulls.index,'currency_exchange_coverage_rate'] = exchanges_list

In [20]:
# NEW VALUE COUNTS 
loans.currency_exchange_coverage_rate.value_counts()

0.1           1096133
0.0            188782
US_dollars     113013
0.2             91964
Name: currency_exchange_coverage_rate, dtype: int64

### VIDEO

Made a simple binary column out of the video_id column and its nulls, thanks to this function.

In [21]:
#FUNCTION
def convertnull(i):
    if np.isnan(i) == True:
        return 0 
    else:
        return 1

In [22]:
#APPLY
loans['has_video'] = loans.video_id.apply(convertnull)

In [23]:
#DROP VIDEO ID COLUMN
loans.drop(columns='video_id',inplace=True)

### EXPIRATION TIME

Expiration time column had some nulls while the posted time didn't. I took the median difference between expiration time and posted time. Using the subset of expiration time nulls, I added that time to each posted_time using timedelta, appending these to a list, and modifying the nulls in the original dataframe using loc and the list. 

In [24]:
#CHANGING DTYPE
loans['posted_time'] = pd.to_datetime(loans['posted_time'])
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])

In [25]:
#SUBSET
expiration_time_nulls = loans[loans['planned_expiration_time'].isnull()]

In [26]:
#MEDIAN TIME
(loans.planned_expiration_time - loans.posted_time).median()

Timedelta('34 days 09:04:48')

In [27]:
#FOR LOOP, LIST CONSTRUCTION
dates = []
for i in expiration_time_nulls.posted_time:
    offset = timedelta(days=34,hours=12,minutes=53)
    dates.append(i + offset)

In [28]:
#CHANGING NULLS USING LOC AND LIST
loans.loc[expiration_time_nulls.index,'planned_expiration_time'] = dates

### LOAN USE AND LENDER TERM

There was some negligeable amounts of nulls in these columns which I dropped.

In [29]:
#DROP NULLS
loans.drop(loans[loans.loan_use.isnull()].index,inplace=True)

In [30]:
#DROP NULLS
loans.drop(loans[loans.lender_term.isnull()].index,inplace=True)

In [31]:
loans.isnull().sum()

unnamed: 0                              0
loan_id                                 0
loan_name                            3558
original_language                       0
description                             0
funded_amount                           0
loan_amount                             0
status                                  0
image_id                                0
activity_name                           0
sector_name                             0
loan_use                                0
country_code                           31
country_name                            0
town_name                               0
currency_policy                         0
currency_exchange_coverage_rate         0
currency                                0
partner_id                              0
posted_time                             0
planned_expiration_time                 0
disburse_time                           0
raised_time                         82287
lender_term                       

# REDUCING VALUE COUNTS AND FEATURE ENGINEERING

Let's formally move to the second part of cleaning: reducing value counts.

### NUMBER OF BORROWERS


The borrower_genders column looked like this, giving the precise amount of individuals per group, so I created a new column number_of_borrowers out of it. 

In [32]:
loans.borrower_genders.value_counts()

female                                                                                                                                                                                                                                                                              995518
male                                                                                                                                                                                                                                                                                315780
female, female                                                                                                                                                                                                                                                                       24387
female, female, female                                                                                                                                 

In [33]:
#LAMBDA FUNCTION
loans['number_of_borrowers'] = loans.borrower_genders.apply(lambda x: len(x.split()))

In [34]:
#NEW COLUMN
loans.number_of_borrowers.value_counts().head(10)

1     1311298
2       31063
3       27533
5       25405
4       22095
6        9395
10       9121
7        8624
8        8042
9        6406
Name: number_of_borrowers, dtype: int64

### BORROWER GENDERS

Looking like that, I knew that I needed to reduce the value counts of borrower_genders. So I thought to be appropriate to have a resulting column with 6 values: male, female, low percentage of group with males, high percentages of group with males, groups of males, groups of females. 

To do this I wrote a function counting the number of males per value in the column and created a new feature num_of_males out of it. Then I created another feature by dividing num_of_males by num_of_borrowers to have the percentage of males per group.

In [35]:
loans.borrower_genders.value_counts()

female                                                                                                                                                                                                                                                                              995518
male                                                                                                                                                                                                                                                                                315780
female, female                                                                                                                                                                                                                                                                       24387
female, female, female                                                                                                                                 

In [36]:
#FUNCTION
def genders(l):
    k = 0
    for i in l.split(','):
        if i == ' male' or i == 'male':
            k += 1
    return k 
      

In [37]:
genders('male, male, female, female')

2

In [38]:
#APPLY
#NEW COLUMN
loans['num_of_males'] = loans.borrower_genders.apply(genders)

In [39]:
loans.num_of_males.value_counts().head(15)

0     1096947
1      342039
2       16004
3       10674
4        7323
5        5518
6        3719
7        2621
8        1767
9        1096
10        762
11        459
12        289
13        191
14        131
Name: num_of_males, dtype: int64

In [40]:
#NEW COLUMN
loans['percentage_of_males'] = loans.num_of_males / loans.number_of_borrowers

In [41]:
loans.percentage_of_males.value_counts()

0.000000    1096947
1.000000     321330
0.500000       9452
0.333333       7123
0.200000       6360
             ...   
0.607143          1
0.422222          1
0.343750          1
0.791667          1
0.677419          1
Name: percentage_of_males, Length: 417, dtype: int64

Looking at the percentage of males column, the loans with 0.000 are loans taken by a female or group of females, the loans 1.0000 are loans taken by a man or group of mans, and the loans with values ranging from 0 to 1 are loans taken by a mixed group. 

I created a custom function accordingly to return a column with males, female, and 2 groups of groups: low percentage and high percentage of groups with males, by taking the mean of the values between 0 and 1.

In [42]:
#MEAN OF VALUES
loans[(loans.percentage_of_males > 0.000001) & (loans.percentage_of_males < 0.999999)].percentage_of_males.mean()

0.37622619452231526

In [43]:
#CUSTOM FUNCTION
def male_percentage(l):
    if l == 0.000000:
        return 'female'
    elif 0.00001 < l < 0.35517323258764366:
        return 'low percentage of male group'
    elif 0.35517323258764366 < l < 0.99999999:
        return 'high percentage of male group'
    else:
        return 'male'


In [44]:
#APPLY
loans['gender_categories'] = loans.percentage_of_males.apply(male_percentage)

In [45]:
loans.gender_categories.value_counts()

female                           1096947
male                              321330
low percentage of male group       37721
high percentage of male group      33879
Name: gender_categories, dtype: int64

The female value takes into account females and groups of females, the male value takes into account males and groups of males. I create 2 other categories by using subsets, pandas and loc on the main dataset.

In [46]:
#SUBSET
males = loans[loans.gender_categories == 'male']      

In [47]:
#LOC
loans.loc[males[males.number_of_borrowers > 1].index,'gender_categories'] = 'group_of_males'

In [48]:
#SUBSET
females = loans[loans.gender_categories == 'female']

In [49]:
#LOC
loans.loc[females[females.number_of_borrowers > 1].index,'gender_categories'] = 'group_of_females'

In [50]:
#5 categories
loans.gender_categories.value_counts()

female                           995518
male                             315780
group_of_females                 101429
low percentage of male group      37721
high percentage of male group     33879
group_of_males                     5550
Name: gender_categories, dtype: int64

In [51]:
loans.drop(columns=['num_of_males','percentage_of_males','borrower_genders'],inplace=True)

I drop the 3 columns that I used to create this final feature as they are redundant or useless.

### TOWNS

Too many values, once again cut down using a custom function. I thought I would keep the first 300 cities and put the rest into a category 'village'.

In [52]:
loans.town_name.value_counts()

El Salvador                         37175
Kaduna                              18444
Lahore                              13614
Senegal                             11704
Eldoret, Rift Valley                10283
                                    ...  
Matlameng, Ha Sekhonyana, Leribe        1
SIMBUCO, TUBOD, LANAO DEL NORTE         1
east poblacion, lopez jaena             1
Kulaa                                   1
Bidii                                   1
Name: town_name, Length: 20502, dtype: int64

In [53]:
#FUNCTION
bigtowns = list(loans.town_name.value_counts().head(300).index)
def towns(i):
    if i in bigtowns:
        return i
    else:
        return 'Village'
loans['town_name'] = loans.town_name.apply(towns)

In [54]:
loans.town_name.value_counts()

Village                     611528
El Salvador                  37175
Kaduna                       18444
Lahore                       13614
Senegal                      11704
                             ...  
Abuyog, Leyte                 1163
Bukavu, Sud Kivu Provice      1148
Mbarara                       1140
Beirut                        1140
Ndunyu Njeru                  1110
Name: town_name, Length: 301, dtype: int64

### BORROWER PICTURED

After looking at these value counts, I decided I wanted four categories out of this : single pictures, groups whose members are all pictured, groups which have more members pictured, and groups which had less members pictured.

I did that first with a custom function dividing the number of "falses", or a person not pictured, with the number of people in the group. 

In [55]:
loans.borrower_pictured.value_counts()

true                                                                                                                                                                                                                           1311298
true, true                                                                                                                                                                                                                       31063
true, true, true                                                                                                                                                                                                                 27533
true, true, true, true, true                                                                                                                                                                                                     25405
true, true, true, true                                                      

In [56]:
#FUNCTION
def pictures(l):
    k = 0
    if (len(l.split()) == 1) & (l == 'true'):
        return 0
    for i in l.split(','):
        if i == ' false' or i == 'false':
            k += 1 
    return k / len(l.split())
      

In [57]:
pictures('false, true, true')

0.3333333333333333

In [58]:
#APPLY
loans.borrower_pictured = loans.borrower_pictured.apply(pictures)

In [59]:
loans.borrower_pictured.value_counts()

0.000000    1478123
0.142857       1447
0.166667       1274
0.125000       1170
0.100000       1047
             ...   
0.075000          1
0.155556          1
0.170732          1
0.140000          1
0.180000          1
Name: borrower_pictured, Length: 130, dtype: int64

I used the same methods as I did with borrower_genders to split this into four categories.

I first created a custom function using the mean of the values between 0 and 1 to divide the groups into groups with less people pictured and groups with more people pictured.

I also knew that a 0.00000 value meant its or a single picture or a group were everyone is pictured, so I made sure to make the distinction, using subsets and loc.


In [60]:
loans[(loans.borrower_pictured > 0.000001) & (loans.borrower_pictured < 0.999999)].borrower_pictured.mean()

0.12366406001981911

In [61]:
#FUNCTION
def pics_percentage(l):
    if l == 0.000000:
        return 'single pic'
    elif 0.00001 < l < 0.11743800516925956:
        return 'group with most pics'
    elif 0.11743800516925956 < l < 0.99999999:
        return 'groups with less pics'

In [62]:
#APPLY
loans['borrower_pictured'] = loans.borrower_pictured.apply(pics_percentage)

In [63]:
loans.borrower_pictured.value_counts()

single pic               1478123
groups with less pics       6555
group with most pics        5199
Name: borrower_pictured, dtype: int64

In [64]:
#SUBSET
sgwithpic = loans[loans.borrower_pictured == 'single pic']

In [65]:
#LOC
loans.loc[sgwithpic[sgwithpic.number_of_borrowers > 1].index,'borrower_pictured'] = 'group with all pics'

In [66]:
# 4 categories
loans.borrower_pictured.value_counts()

single pic               1311298
group with all pics       166825
groups with less pics       6555
group with most pics        5199
Name: borrower_pictured, dtype: int64

### DAYS OF CAMPAIGN DURATION

I also realised I could feature engineer a column displaying the number of days a certain loan was on campaign. I did that by taking the difference between the expired time and posted time. Then I took specfically the days using datetime.

In [67]:
loans['posted_time'] = pd.to_datetime(loans['posted_time'])
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
#NEW COLUMN
loans['campaign_duration'] = loans.planned_expiration_time - loans.posted_time

In [68]:
loans.campaign_duration.value_counts()

34 days 12:53:00    149953
30 days 00:09:41        24
30 days 00:05:00        24
30 days 00:09:31        24
30 days 00:05:35        23
                     ...  
53 days 11:38:38         1
39 days 21:27:03         1
49 days 22:06:16         1
36 days 07:34:55         1
42 days 15:46:22         1
Name: campaign_duration, Length: 816302, dtype: int64

In [69]:
#DATETIME
loans['days_of_campaign_duration'] = loans.campaign_duration.dt.days

In [70]:
loans.drop(columns = 'campaign_duration', inplace=True)

In [71]:
loans.days_of_campaign_duration.value_counts()

34     280083
30     181571
32     150468
33     144663
31     141230
        ...  
141         2
143         2
116         1
113         1
146         1
Name: days_of_campaign_duration, Length: 109, dtype: int64

##  THE TARGET VARIABLE - STATUS

Let's have a look at the value counts of the target variable.

In [72]:
loans.status.value_counts(normalize=True)

funded         0.944769
expired        0.051137
fundRaising    0.004094
Name: status, dtype: float64

I dropped the loans still fundraising as we still can't learn from these recent loans, leaving this a binary classification problem. 

In [73]:
loans.drop(loans[loans.status == 'fundRaising'].index,inplace=True)

In [74]:
loans.status.value_counts(normalize=True)

funded     0.948653
expired    0.051347
Name: status, dtype: float64

Since I will first do some Exploratory Data Analysis, I will set the 'funded' majority class as the positive value to display percentages funded. However, once I've finished with my EDA, I will set the minority 'expired' class as positive for the modelling.

In [75]:
loans['status'] = loans.status.map(lambda x: 0 if 'expired' in x else 1)

In [76]:
loans.status.value_counts(normalize=True)

1    0.948653
0    0.051347
Name: status, dtype: float64

## REMOVING USELESS COLUMNS

* I remove loan_id, image_id, loan_name and borrower_names as these columns don't add much to the analysis
- I remove country_code due to its redudancy with country_name

In [77]:
loans.drop(columns=['loan_id','image_id','loan_name', 'borrower_names',
                    'country_code','disburse_time','num_lenders_total',
                    'raised_time','funded_amount''num_journal_entries',
                    'num_bulk_entries'],inplace=True)

In [78]:
loans.drop(columns=['num_lenders_total','raised_time','funded_amount'],inplace=True)

In [79]:
#DROP
loans.drop(columns=['num_journal_entries','num_bulk_entries'],inplace=True)

In [80]:
loans.drop(columns=['unnamed: 0'],inplace=True)

In [81]:
loans.loc[loans[loans.tags.isnull()].index,'tags'] = 'unspecified'
loans.drop(loans[loans.delinquency.isnull()].index,inplace=True)
loans.drop(loans[loans.default_rate.isnull()].index,inplace=True)
loans.loc[loans[loans.rural_pct.isnull()].index,'rural_pct'] = 0.0

In [87]:
loans.isnull().sum()

original_language                  0
description                        0
loan_amount                        0
status                             0
activity_name                      0
sector_name                        0
loan_use                           0
country_name                       0
town_name                          0
currency_policy                    0
currency_exchange_coverage_rate    0
currency                           0
partner_id                         0
posted_time                        0
planned_expiration_time            0
lender_term                        0
tags                               0
borrower_pictured                  0
repayment_interval                 0
distribution_model                 0
time                               0
delinquency                        0
default_rate                       0
rural_pct                          0
has_video                          0
number_of_borrowers                0
gender_categories                  0
d

In [88]:
loans.to_csv('loans2_data_cleaned.csv')